In [1]:
import youtube_dl
import sys
from requests import get

# URLs = [
#     'https://tv.naver.com/v/36179013', #test
#     'https://tv.naver.com/v/36072694', #source
#     'https://tv.naver.com/v/36073744', #highlight
# ]

class Downloader:
    def __init__(self, resolution):
        self.resolution = resolution
    
    def get_url(self ,url):
        result = ""
        ydl_opts = {
            # 'format': 'bestvideo/best', # best resolution?
        }
        ydl = youtube_dl.YoutubeDL(ydl_opts)
        info_dict = ydl.extract_info(url, download=False)
        print(info_dict)
        formats = info_dict.get('formats',None)
        for f in formats:
            if f.get('format_id', None) == 'avc1_' + self.resolution + 'P':
                result = f.get('url', None)
        print("\n\nresult >> ", result)
        return result
    
    def download(self, url, file_name):
        dl_url = self.get_url(url)
        with open(file_name + '.mp4', "wb") as file:   
            response = get(dl_url)
            print(response)
            file.write(response.content)

In [2]:
resolution = '720'
dl = Downloader(resolution)
file_name = "video_data.txt"

with open(file_name, "r") as f:
    for line in f:
        url, video_name, highlight_url = line.strip().split(",")
        dl.download(url, video_name)
        dl.download(highlight_url, video_name+"_HL")

[Naver] 36072694: Downloading JSON metadata
[Naver] 36072694: Downloading JSON metadata
{'id': '36072694', 'title': 'KT-LG 풀영상', 'formats': [{'format_id': 'HLS_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_ALPHA/navertv_2023_05_18_1324/hls/13c12180-f57b-11ed-8f71-a0369ffdda14.m3u8?__gda__=1685305598_0f491dd085b2f947819eb689e7767134', 'width': 256, 'height': 144, 'vbr': 83, 'abr': 64, 'filesize': None, 'protocol': 'm3u8_native', 'tbr': 147, 'ext': 'm3u8', 'format': 'HLS_144P - 256x144', 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3699.0 Safari/537.36', 'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'en-us,en;q=0.5'}}, {'format_id': 'avc1_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_ALPHA/navertv_2023_05_18_1909/distribution_engi

<Response [200]>
[Naver] 36073744: Downloading JSON metadata
[Naver] 36073744: Downloading JSON metadata
{'id': '36073744', 'title': "[전체HL] '박동원 3타점+5회 6득점' LG, KT 꺾고 2연승", 'formats': [{'format_id': 'HLS_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_ALPHA/navertv_2023_05_18_1498/hls/fd25238d-f57d-11ed-a376-a0369ffac3bc.m3u8?__gda__=1685291142_430a51484d6d2269c8082895fd22ce9c', 'width': 256, 'height': 144, 'vbr': 81, 'abr': 64, 'filesize': None, 'protocol': 'm3u8_native', 'tbr': 145, 'ext': 'm3u8', 'format': 'HLS_144P - 256x144', 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3699.0 Safari/537.36', 'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'en-us,en;q=0.5'}}, {'format_id': 'avc1_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_AL

<Response [200]>


In [ ]:
import cv2
import sys

class Video_capture:
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.ret = None 
        self.frame = None 
        
    def move_frame(self, frame_cnt):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_cnt)
        self.read()
        
    def make_grayscale(self):
        return cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
    
    def read(self):
        self.ret, self.frame = self.cap.read()
        return

class Highlight_finder:
    def __init__(self, source, highlight, precision):
        self.name = source[:-4]
        self.source = Video_capture(source)
        self.highlight= Video_capture(highlight)
        self.frame_cnt = 0
        self.highlight_frame_cnt = 210 # 네이버 로고 지난 프레임
        self.precision = precision
        self.result = []
    
    def load_video(self):
        if not self.source.cap.isOpened():
                print('source video not opened')
                exit(-1)
        if not self.highlight.cap.isOpened():
                print('highlight video not opened')
                exit(-1)
        self.source.move_frame(self.frame_cnt)
        self.highlight.move_frame(self.highlight_frame_cnt)       
        return
    
    def match_template(self, src, frame):
        R = cv2.matchTemplate(src, frame, cv2.TM_CCORR_NORMED)
        _, maxVal, _, maxLoc = cv2.minMaxLoc(R)
        return maxVal
    
    def find_highlight(self):
        start = 0
        end = 0
        f = open(self.name +".txt",'w')
        while True: # 영상이 끝날때까지
            self.frame_cnt += 5
            if self.frame_cnt % 1800 == 0:
                print(self.frame_cnt / 1800, "min", self.highlight_frame_cnt)
            for i in range(5):
                self.source.read()
            if not self.source.ret or not self.highlight.ret:
                break
        #---------------------------------------------------------------------
            maxVal = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())

            #cv2.imshow('source', self.source.frame)
            key = cv2.waitKey(25)
            if key == 27:
                cv2.destroyAllWindows()
                break

            if maxVal < self.precision: # 영상이 일치하는지 체크
                continue
            else: # 일치한다면
                result = maxVal
                if(self.frame_cnt - end > 150):
                    if end != 0 and end - start >= 60:
                        start_time = '%d:%d:%d' % (start // 29.97 // 3600, start//29.97%3600//60, start//29.97%60)
                        end_time = '%d:%d:%d' % (end // 29.97 // 3600, end//29.97%3600//60 , end//29.97%60)
                        f.write(str(start) + "," + str(end)+", " + start_time + "," + end_time+"\n")
                    start = self.frame_cnt
                while True: # 일치하지 않는 구간까지 반복
                    print(self.frame_cnt, result, self.highlight_frame_cnt)
                    self.highlight_frame_cnt += 15
                    self.frame_cnt += 15
                    for i in range(15):
                        self.highlight.read()
                        self.source.read()
                        
                    '''
                    cv2.imshow('source', self.source.frame)
                    cv2.imshow('highlight', self.highlight.frame)
                    key = cv2.waitKey(25)
                    if key == 27:
                        cv2.destroyAllWindows()
                    '''
                    if not self.source.ret or not self.highlight.ret: 
                        break
                    result = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())
                    if result > self.precision: # 일치한다면
                        end = self.frame_cnt
                        continue
                    else: # 일치하지 않는다면
                        print(self.frame_cnt, result, self.highlight_frame_cnt)
                        end = self.frame_cnt - 15
                        self.highlight_frame_cnt += 30
                        for i in range(30):
                            self.highlight.read()
                        break
                
        #----------------------------------------------------------------------
        if self.source.cap.isOpened():
            self.source.cap.release()

        if self.highlight.cap.isOpened():
            self.highlight.cap.release()

        #cv2.destroyAllWindows()
        print("END")
        return
